In [1]:
import numpy as np
from catboost import CatBoostRegressor, Pool
import datetime
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
import optuna
import pandas as pd
from optuna.samplers import TPESampler
import seaborn as sns
from sklearn.compose import ColumnTransformer
from sklearn.decomposition import PCA
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.preprocessing import OneHotEncoder , OrdinalEncoder, StandardScaler

In [24]:
y_train = pd.read_csv('y_train.csv')

In [23]:
y_test = pd.read_csv('y_test.csv')

In [26]:
print(y_train.shape), print(y_test.shape)

(6552, 1)
(2808, 1)


(None, None)

In [28]:
df_train = pd.read_csv('X_train_clean_F_R.csv')

In [29]:
df_test = pd.read_csv('X_test_clean_F_R.csv')

In [30]:
df_test.head(3)

,Category,Reviews,Size,Installs,Content Rating,Current Ver,Android Ver,Main_Genre,Days_Since_Last_Update
0,BUSINESS,25195,21000.0,1000000,Everyone,v3.4.0-3,4.0.3 and up,Business,2015
1,PHOTOGRAPHY,351254,4000.0,10000000,Everyone,4.6.0.0,4.2 and up,Photography,1892
2,GAME,112565,32000.0,1000000,Teen,1.1.97,4.4 and up,Action,1870


In [31]:
ord_cat_pipeline = Pipeline(
    [("OrdEncoder", OrdinalEncoder())
    ]
)

num_pipeline = Pipeline(
    [("SScaler", StandardScaler())
    ]
)

In [32]:
preprocessing = ColumnTransformer(
    [
     ("Ord_Process_Categorical", ord_cat_pipeline, ["Category","Content Rating","Current Ver","Android Ver","Main_Genre"]),
     ("Process_Num_Scaler", num_pipeline, ["Reviews","Size","Installs","Days_Since_Last_Update"]),
    ], remainder = "passthrough")

In [33]:
pipe_preprocessed_train = preprocessing.fit_transform(df_train)

In [34]:
df_pipe_train = pd.DataFrame(pipe_preprocessed_train, columns= preprocessing.get_feature_names_out())

In [35]:
pipe_preprocessed_test = preprocessing.fit_transform(df_test)

In [36]:
df_pipe_test = pd.DataFrame(pipe_preprocessed_test, columns= preprocessing.get_feature_names_out())

In [37]:
print(df_pipe_train.shape), print(df_pipe_test.shape)

(6552, 9)
(2808, 9)


(None, None)

In [38]:
from xgboost import XGBRegressor

In [41]:
xgb_param = {
          'max_depth': [3, 6, 10],
          'learning_rate': [0.01, 0.1, 0.4],
          'subsample': np.arange(0.5, 1.0, 0.1),
          'n_estimators': [100, 500, 750],
}

cv = 5

gs_xgb = GridSearchCV(XGBRegressor(),
                        xgb_param,
                        cv=cv,
                        scoring='neg_mean_absolute_error',
                        verbose=1,
                        n_jobs=-1)

In [42]:
gs_xgb.fit(df_pipe_train, y_train)

Fitting 5 folds for each of 135 candidates, totalling 675 fits


GridSearchCV(cv=5,
             estimator=XGBRegressor(base_score=None, booster=None,
                                    callbacks=None, colsample_bylevel=None,
                                    colsample_bynode=None,
                                    colsample_bytree=None,
                                    early_stopping_rounds=None,
                                    enable_categorical=False, eval_metric=None,
                                    feature_types=None, gamma=None, gpu_id=None,
                                    grow_policy=None, importance_type=None,
                                    interaction_constraints=None,
                                    learning_rate=None, m...
                                    max_depth=None, max_leaves=None,
                                    min_child_weight=None, missing=nan,
                                    monotone_constraints=None, n_estimators=100,
                                    n_jobs=None, num_parallel_tree=None,
                                    predictor=None, random_state=None, ...),
             n_jobs=-1,
             param_grid={'learning_rate': [0.01, 0.1, 0.4],
                         'max_depth': [3, 6, 10],
                         'n_estimators': [100, 500, 750],
                         'subsample': array([0.5, 0.6, 0.7, 0.8, 0.9])},
             scoring='neg_mean_absolute_error', verbose=1)

In [43]:
y_pred=gs_xgb.predict(df_pipe_test)

In [44]:
print(mean_absolute_error(y_test, y_pred))
print(mean_squared_error(y_test, y_pred))

0.3347787946717352
0.2690343156102595
